In [1]:
from src import functions as f
import pandas as pd

In [62]:
f.load_pickle_file("pages_list_nodes", dir_path = "./data/outputs/")

['/',
 '/search/all',
 '/view-prove-immigration-status',
 '/prove-right-to-work',
 '/browse/working',
 '/find-a-job',
 '/check-state-pension',
 '/check-state-pension/sign-in/prove-identity',
 '/jobseekers-allowance',
 '/jobseekers-allowance/eligibility',
 '/browse/working/state-pension',
 '/view-right-to-work',
 '/contact-jobcentre-plus',
 '/government/organisations/disclosure-and-barring-service',
 '/request-copy-criminal-record',
 '/topic/further-education-skills/apprenticeships',
 '/apply-apprenticeship',
 '/browse/working/finding-job',
 '/jobseekers-allowance/apply-new-style-jsa',
 '/dbs-update-service',
 '/criminal-record-checks-apply-role',
 '/new-state-pension',
 '/government/organisations/department-for-work-pensions',
 '/apply-national-insurance-number',
 '/state-pension-age',
 '/contact-pension-service',
 '/new-state-pension/what-youll-get',
 '/browse/benefits',
 '/check-job-applicant-right-to-work',
 '/contact-jobcentre-plus/new-benefit-claims',
 '/become-apprentice/apply-fo

In [2]:
# Economic recovery pages

G, A, T = f.get_er_graph("./data/inputs/functional_session_hit_directed_graph_er.gpickle")
ner_df = pd.read_csv("./data/inputs/ner_onehot.csv")

C:\Users\jbryb\virual_envs_python\pytorch_env\lib\site-packages\networkx\linalg\graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)
C:\Users\jbryb\OneDrive\Documents\Projects\github_gov_newtork_jrad\src\functions.py:45: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  T = nx.adjacency_matrix(G, weight="edgeWeight").todense()
C:\Users\jbryb\OneDrive\Documents\Projects\github_gov_newtork_jrad\src\functions.py:52: RuntimeWarning: invalid value encountered in divide
  T_probs = T_array / sum_of_rows[:, np.newaxis]  # increase dimension for broadcasting


Max row sum of probs: 1.0
Max row sum of probs: 1.0


C:\Users\jbryb\AppData\Local\Temp\ipykernel_15024\717052584.py:4: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  ner_df = pd.read_csv("./data/inputs/ner_onehot.csv")


In [3]:
# Write to pickle basiv graph matrices

f.dump_pickle_file(G, "Graph_er", dir_path="./data/outputs")
f.dump_pickle_file(A, "Adjacency_er", dir_path="./data/outputs")
f.dump_pickle_file(T, "Transition_er", dir_path="./data/outputs")

()

In [4]:
# Now load from pickles using a function in src folder

G2 = f.load_pickle_file("Graph_er", dir_path="./data/outputs")
A2 = f.load_pickle_file("Adjacency_er", dir_path="./data/outputs")
T2 = f.load_pickle_file("Transition_er", dir_path="./data/outputs")

In [5]:
print("Number of nodes of the graph", G.number_of_nodes())

Number of nodes of the graph 10631


In [6]:
import networkx as nx

# In a single object:

# Create grpah from transition matrix
GGG2 = nx.from_numpy_array(T2, create_using = nx.DiGraph)

# GGG2.nodes = G2.nodes

In [41]:
ner_df = pd.read_csv("./data/inputs/ner_onehot.csv")

C:\Users\jbryb\AppData\Local\Temp\ipykernel_11344\2202456393.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  ner_df = pd.read_csv("./data/inputs/ner_onehot.csv")


In [7]:
ner_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,page_index_10 years,page_index_5 years,page_index_abroad,page_index_academics,page_index_academies,page_index_academy,page_index_academy trust,...,page_index_world health organization,page_index_writing,page_index_year,page_index_years,page_index_young people,page_index_young person,page_index_youth,page_index_youth court,page_index_youth mobility scheme,page_index_zimbabwe
0,index,page_index,page_link \ word index,0.0,1.0,2.0,3.0,4.0,5.0,6.0,...,1545.0,1546.0,1547.0,1548.0,1549.0,1550.0,1551.0,1552.0,1553.0,1554.0
1,0,0,https://www.gov.uk/view-prove-immigration-status,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,https://www.gov.uk/prove-right-to-work,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,3,https://www.gov.uk/find-a-job,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,5,https://www.gov.uk/check-state-pension/sign-in...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# ner_df.head()
ner_df.drop(['Unnamed: 0', 'Unnamed: 1'], axis = 1, inplace = True) #.columns
ner_df["Unnamed: 2"] = ner_df["Unnamed: 2"].str.replace('https://www.gov.uk' , '')
ner_df.rename(columns = {"Unnamed: 2": "page"}, inplace = True)
ner_df.set_index("page", inplace = True)
ner_df.drop('page_link \\ word index', inplace = True)
ner_df.head()

C:\Users\jbryb\AppData\Local\Temp\ipykernel_15024\1234686198.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ner_df["Unnamed: 2"] = ner_df["Unnamed: 2"].str.replace('https://www.gov.uk' , '')


,page_index_10 years,page_index_5 years,page_index_abroad,page_index_academics,page_index_academies,page_index_academy,page_index_academy trust,page_index_acas,page_index_accident,page_index_accommodation,...,page_index_world health organization,page_index_writing,page_index_year,page_index_years,page_index_young people,page_index_young person,page_index_youth,page_index_youth court,page_index_youth mobility scheme,page_index_zimbabwe
page,,,,,,,,,,,,,,,,,,,,,
/view-prove-immigration-status,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/prove-right-to-work,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/find-a-job,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/check-state-pension/sign-in/prove-identity,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/jobseekers-allowance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
node_relabel_dict = dict()
node_name_list = list( G2.nodes )

for i in range( len(list(GGG2.nodes)) ):
    node_relabel_dict[i] = node_name_list[i]
    
GGG2 = nx.relabel_nodes(GGG2, mapping = node_relabel_dict)
GGG2_features = GGG2

In [12]:
# Add node features to the graph object

omitted_nodes = []

# empty_feature vector for pages without any named entities
empty_feature_vector = ner_df.iloc[0, :].copy()
empty_feature_vector.iloc[:] = 0

for node in GGG2_features.nodes():
    try:
        GGG2_features.nodes[node]["node_feature"] = ner_df.loc[node, :]
    except:
        GGG2_features.nodes[node]["node_feature"] = empty_feature_vector

        omitted_nodes.append(node)

# Drop webpages without 
GGG2_features_nozeros = GGG2_features
GGG2_features_nozeros.remove_nodes_from(omitted_nodes)

In [17]:
T2[0, 1]

0.24342119512247948

In [70]:
# GGG2_features = GGG2

# # Create a dummy feature matrix
# node_features = np.eye( len(list(GGG2_features.nodes)), len(list(GGG2_features.nodes)) )

# for i, node in enumerate(GGG2_features.nodes()):
#     print(node)
# #     GGG2_features.nodes[node]["node_feature"] = node_features[i,:]

In [18]:
T2[1, 0]

0.07841605329166496

In [74]:
(T2[:10, :10]).toarray()

array([[0.00000000e+00, 2.43421195e-01, 8.79339845e-03, 2.24166685e-02,
        2.20051548e-01, 1.81499209e-01, 9.09661909e-04, 7.36392974e-04,
        1.02445258e-02, 1.51610318e-04],
       [7.84160533e-02, 0.00000000e+00, 5.41140672e-02, 5.49775895e-02,
        6.57921790e-03, 7.64011678e-02, 1.27061146e-02, 1.23360336e-04,
        1.35696369e-02, 9.45762572e-04],
       [2.71350442e-02, 2.04038704e-02, 0.00000000e+00, 7.56275417e-01,
        6.31047539e-04, 3.50581966e-04, 7.01163932e-05, 0.00000000e+00,
        1.40232786e-04, 0.00000000e+00],
       [8.20012260e-02, 2.15494884e-02, 1.81025133e-01, 0.00000000e+00,
        4.00811053e-03, 1.88616966e-04, 9.43084830e-05, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [6.82529405e-02, 1.43520017e-02, 1.02514298e-03, 2.26610554e-03,
        0.00000000e+00, 2.58983490e-03, 3.23729362e-04, 2.69774469e-04,
        2.69774469e-04, 5.39548937e-05],
       [2.26935123e-01, 4.80536913e-02, 6.26398210e-04, 8.05369128e-04,
   

In [75]:
print("Is the new graph directed:", nx.is_directed(GGG2_renamed))

Is the new graph directed: True


In [68]:
GGG2_features_renamed.remove_nodes_from(omitted_nodes)

In [78]:
GGG2_renamed.get_edge_data('/','/search/all')

{'weight': 0.24342119512247948}

In [79]:
GGG2_renamed.get_edge_data('/search/all','/')

{'weight': 0.07841605329166496}

In [80]:
f.dump_pickle_file(GGG2_renamed, "Graph_er_weighted", dir_path="./data/outputs")

()

In [81]:
f.dump_pickle_file(GGG2_features_renamed, "Graph_er_weighted_wfeatures", dir_path="./data/outputs")

()

In [ ]:
# The nx.from_numpy_array idea
# https://stackoverflow.com/questions/62375834/how-can-i-add-weight-to-networkx-edges